In [14]:
import tensorflow as tf
import numpy as np

class Autoencoder:
    
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch                                                 
        self.learning_rate = learning_rate                                 

        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])      

        with tf.name_scope('encode'):                                      
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):                                      
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases

        self.x = x                                                         
        self.encoded = encoded                                             
        self.decoded = decoded                                             

        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))                                                      
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)     
        self.saver = tf.train.Saver()
    
    def get_batch(self, X, size):
        a = np.random.choice(len(X), size, replace=False)
        return X[a]
    
    def train(self, data, batch_size=10):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(500):                                  
                    batch_data = self.get_batch(data, batch_size)     
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: batch_data})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
                
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt')                       
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})                                          
            print('input', data)
            print('compressed', hidden)
            print('reconstructed', reconstructed)
            return reconstructed

In [15]:
from sklearn import datasets

hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

epoch 0: loss = 3.3524835109710693
epoch 10: loss = 0.4601932466030121
epoch 20: loss = 0.31722983717918396
epoch 30: loss = 0.3636620044708252
epoch 40: loss = 0.25878340005874634
epoch 50: loss = 0.2544558644294739
epoch 60: loss = 0.3079870641231537
epoch 70: loss = 0.2487819641828537
epoch 80: loss = 0.20135048031806946
epoch 90: loss = 0.23408304154872894
epoch 100: loss = 0.26166799664497375
epoch 110: loss = 0.29344698786735535
epoch 120: loss = 0.3088926374912262
epoch 130: loss = 0.21854114532470703
epoch 140: loss = 0.3401826024055481
epoch 150: loss = 0.26538515090942383
epoch 160: loss = 0.3017844557762146
epoch 170: loss = 0.434401273727417
epoch 180: loss = 0.2707311511039734
epoch 190: loss = 0.38534414768218994
epoch 200: loss = 0.2851945459842682
epoch 210: loss = 0.2501869797706604
epoch 220: loss = 0.3079162538051605
epoch 230: loss = 0.21453450620174408
epoch 240: loss = 0.3212789297103882
INFO:tensorflow:Restoring parameters from ./model.ckpt
input [[8, 4, 6, 2]]
c

array([[6.8943553, 2.7975569, 6.2872977, 2.2577817]], dtype=float32)